# Time-explicit LCA of an electric vehicle


This notebook shows how to use `bw_timex` with a cradle-to-grave case study of an electric vehicle (ev). The case study is simplified, not meant to reflect the complexity of electric mobility but to demonstrate hot to use `bw_timex`. 

More information on the inner workings of `bw_timex` can be found [here](https://timex.readthedocs.io/en/latest/content/theory.html).


> **Note:** This is the "premise" version of this notebook that works with ecoinvent and premise data. If you don't have access to that, please check out the ["standalone" version](https://github.com/brightway-lca/bw_timex/blob/main/notebooks/example_electric_vehicle_standalone.ipynb) of this notebook.

In [36]:
import bw2data as bd

bd.projects.set_current("multi-level-tds")

In [37]:
biosphere = bd.Database("biosphere")
biosphere.write(
    {
        ("biosphere", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    }
)

if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.register()

if "background_2020" in bd.databases:
    del bd.databases["background_2020"] # to make sure we create the foreground from scratch
background_2020 = bd.Database("background_2020")
background_2020.register()

if "background_2030" in bd.databases:
    del bd.databases["background_2030"] # to make sure we create the foreground from scratch
background_2030 = bd.Database("background_2030")
background_2030.register()

100%|██████████| 1/1 [00:00<00:00, 23831.27it/s]

Vacuuming database 


In [38]:
node_co2 = biosphere.get("CO2")

fu = foreground.new_node("fu", name="fu", unit="unit")
fu['reference product'] = "fu"
fu.save()

first_level_input = foreground.new_node("first_level_input", name="first_level_input", unit="unit")
first_level_input['reference product'] = "first_level_input"
first_level_input.save()

background_process_2020 = background_2020.new_node("background_process", name="background_process", unit="background_process")
background_process_2020['reference product'] = "background_process"
background_process_2020.save()

background_process_2030 = background_2030.new_node("background_process", name="background_process", unit="background_process")
background_process_2030['reference product'] = "background_process"
background_process_2030.save()


In [39]:
fu.new_edge(input=fu, amount=1, type="production").save()
first_level_input_to_fu = fu.new_edge(input=first_level_input, amount=1, type="technosphere")

first_level_input.new_edge(input=first_level_input, amount=1, type="production").save()
background_process_2020_to_first_level_input = first_level_input.new_edge(input=background_process_2020, amount=1, type="technosphere")

background_process_2020.new_edge(input=background_process_2020, amount=1, type="production").save()
background_process_2020.new_edge(input=node_co2, amount=17, type="biosphere").save()

background_process_2030.new_edge(input=background_process_2030, amount=1, type="production").save()
background_process_2030.new_edge(input=node_co2, amount=11, type="biosphere").save()


In [40]:
from bw_temporalis import TemporalDistribution
import numpy as np

td_first_level_input_to_fu = TemporalDistribution(
    date=np.array([1, 2], dtype="timedelta64[Y]"), amount=np.array([0.2, 0.8])
)

td_background_process_2020_to_first_level_input = TemporalDistribution(
    date=np.array([2, 3], dtype="timedelta64[Y]"), amount=np.array([0.5, 0.5])
)

In [41]:
first_level_input_to_fu["temporal_distribution"] = td_first_level_input_to_fu
first_level_input_to_fu.save()

background_process_2020_to_first_level_input["temporal_distribution"] = td_background_process_2020_to_first_level_input
background_process_2020_to_first_level_input.save()

## LCA using `bw_timex`


As usual, we need to select a method:

In [42]:
method = ("GWP", "example")
bd.Method(method).write(
    [
        (("biosphere", "CO2"), 1),
    ]
)

In [43]:
from datetime import datetime

database_date_dict = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "background_2030": datetime.strptime("2030", "%Y"),
    "foreground": "dynamic", # flag databases that should be temporally distributed with "dynamic"
}

In [44]:
from bw_timex import TimexLCA

tlca = TimexLCA({fu: 1}, method, database_date_dict)

In [45]:
tlca.build_timeline(temporal_grouping="month")

Starting graph traversal
Calculation count: 2


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:195: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(


,date_producer,producer_name,date_consumer,consumer_name,amount,interpolation_weights
0,2024-08-01,fu,2024-08-01,-1,1.0,None
1,2025-08-01,first_level_input,2024-08-01,fu,0.2,None
2,2026-08-01,first_level_input,2024-08-01,fu,0.8,None
3,2027-08-01,background_process,2025-08-01,first_level_input,0.5,"{'background_2020': 0.24199288256227758, 'back..."
4,2028-08-01,background_process,2025-08-01,first_level_input,0.5,"{'background_2020': 0.14180125923898168, 'back..."
5,2028-08-01,background_process,2026-08-01,first_level_input,0.5,"{'background_2020': 0.14180125923898168, 'back..."
6,2029-08-01,background_process,2026-08-01,first_level_input,0.5,"{'background_2020': 0.041883383520394224, 'bac..."
